- **What it is:**  
  - A script to fetch historical performance data for REIT benchmarks from Yahoo Finance, including their inception dates.

- **What it does:**  
  - Connects to Snowflake to get benchmark symbols.  
  - Downloads and normalizes historical price data for each benchmark.  
  - Retrieves or estimates each benchmark’s inception date.  
  - Prepares a cleaned dataset for further analysis or upload.

- **Why it’s important:**  
  - Provides accurate and complete benchmark data for portfolio performance evaluation.  
  - Ensures benchmark history includes inception dates for context.  
  - Supports reliable reporting and analysis in real estate investment management.


# **Benchmark Performance Table**

## Establishing Snowflake Connection

You must download the env.txt file which is located in my GDrive folder ("Shiv-Code") and upload using the "Choose Files" widget after running the below cell.

In [2]:
from google.colab import files
uploaded = files.upload()

Saving env.txt to env.txt


In [3]:
import os

if os.path.exists("env.txt"):
    os.rename("env.txt", "env")
    print("Renamed env.txt to env")
else:
    print("File not found. Make sure you uploaded env.txt.")

Renamed env.txt to env


In [4]:
!pip install snowflake-connector-python python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.6 MB/s eta 0:00:00


In [5]:
from dotenv import load_dotenv
load_dotenv("env")  # This loads the Snowflake credentials into the environment

True

In [6]:
import snowflake.connector
import os

# Pull in credentials
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
)

# Run a simple test query
cur = conn.cursor()
cur.execute("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_DATABASE();")
for row in cur:
    print(row)

cur.close()
conn.close()

('SHIVNEETN', 'AST_REALESTATE_DB_RW', 'AST_REALESTATE_DB')


## Connect to BGI to pull Benchmark Codes

In [1]:
pip install yfinance

In [7]:
import yfinance as yf
import pandas as pd
from snowflake.connector import connect
import os

def fetch_benchmark_performance_with_inception():
    """
    Fetches benchmark performance data for REIT benchmarks from Yahoo Finance, including inception dates,
    then prepares the data for upload.

    Steps:
    1. Connects to Snowflake and retrieves REIT benchmark codes and symbols from BenchmarkGeneralInformation.
    2. Defines the historical data range from 2022-01-01 to today.
    3. For each benchmark symbol:
       - Cleans the symbol for Yahoo Finance usage.
       - Attempts to retrieve the inception date from ticker info keys such as 'startDate' or 'firstTradeDate'.
         If unavailable, uses the earliest available historical price date as a fallback.
       - Downloads historical price data within the date range.
       - Normalizes the price series to start at 100 for relative performance.
       - Populates static columns such as BENCHMARKCODE, currency info, and performance frequency.
       - Sets the HISTORYDATE column to the inception date for all rows.
       - Adds HISTORYDATE1 as the date component of the price date.
    4. Concatenates all individual benchmark data into one DataFrame.

    Returns:
        pd.DataFrame: Concatenated DataFrame of benchmark performance data ready for upload.
    """
    # Connect to Snowflake
    conn = connect(
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        role=os.getenv("SNOWFLAKE_ROLE"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        database="AST_REALESTATE_DB",
        schema="DBO"
    )

    # Query to get REIT benchmarks with valid symbols
    bgi_df = pd.read_sql("""
        SELECT BENCHMARKCODE, SYMBOL
        FROM BenchmarkGeneralInformation
        WHERE BENCHMARKCODE LIKE 'REITBENCH%'
          AND SYMBOL IS NOT NULL
    """, conn)

    # Define date range for historical data
    start_date = "2022-01-01"
    end_date = pd.Timestamp.today().strftime("%Y-%m-%d")

    rows = []

    for _, row in bgi_df.iterrows():
        symbol_raw = row["SYMBOL"]
        symbol = symbol_raw.replace("$", "").strip()
        code = row["BENCHMARKCODE"]

        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            # Attempt to extract inception date from info
            inception_date = None
            if "startDate" in info and info["startDate"]:
                inception_date = pd.to_datetime(info["startDate"], unit='s')
            elif "firstTradeDate" in info and info["firstTradeDate"]:
                inception_date = pd.to_datetime(info["firstTradeDate"], unit='s')
            else:
                # Fallback to earliest price date if inception date is unavailable
                hist = ticker.history(start=start_date, end=end_date)
                if hist.empty:
                    print(f"No data for {symbol}")
                    continue
                inception_date = hist.index.min()

            # Fetch historical price data
            hist = ticker.history(start=start_date, end=end_date)
            if hist.empty:
                print(f"No historical data for {symbol}")
                continue
            hist = hist.reset_index()

            # Normalize values to 100 at start
            if "Adj Close" in hist.columns:
                base_price = hist["Adj Close"].iloc[0]
                hist["VALUE"] = (hist["Adj Close"] / base_price) * 100
            elif "Close" in hist.columns:
                base_price = hist["Close"].iloc[0]
                hist["VALUE"] = (hist["Close"] / base_price) * 100
            else:
                print(f"No 'Adj Close' or 'Close' data for {symbol}")
                continue

            # Populate static metadata columns
            hist["BENCHMARKCODE"] = code
            hist["PERFORMANCEDATATYPE"] = "Total Return"
            hist["CURRENCY"] = "US Dollar"
            hist["CURRENCYCODE"] = "USD"
            hist["PERFORMANCEFREQUENCY"] = "Daily"

            # Set HISTORYDATE column to inception date for all rows
            hist["HISTORYDATE"] = inception_date

            # Extract date only for HISTORYDATE1
            hist["HISTORYDATE1"] = hist["Date"].dt.date

            # Append relevant columns to rows list
            rows.append(hist[[
                "BENCHMARKCODE", "PERFORMANCEDATATYPE", "CURRENCY", "CURRENCYCODE",
                "PERFORMANCEFREQUENCY", "HISTORYDATE", "HISTORYDATE1", "VALUE"
            ]])

            print(f"Fetched data for {symbol} → {code}")

        except Exception as e:
            print(f"Error for {symbol} → {e}")

    # Combine all benchmark data into one DataFrame or return empty if none
    if rows:
        bp_df = pd.concat(rows, ignore_index=True)
    else:
        print("No benchmark data returned.")
        bp_df = pd.DataFrame()

    return bp_df

# Usage:
# bp_df = fetch_benchmark_performance_with_inception()
# print(bp_df)


/tmp/ipython-input-1945075176.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bgi_df = pd.read_sql("""
ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:$BBREIT: possibly delisted; no timezone found
ERROR:yfinance:HTTP Error 404: 


No data for BBREIT


ERROR:yfinance:$FNREI: possibly delisted; no timezone found


No data for FNREI


ERROR:yfinance:$RMZ: possibly delisted; no price data found  (1d 2022-01-01 -> 2025-08-01)


No data for RMZ
Fetched data for SCHH → REITBENCH03
Fetched data for VNQ → REITBENCH01
Fetched data for IYR → REITBENCH02
     BENCHMARKCODE PERFORMANCEDATATYPE   CURRENCY CURRENCYCODE  \
0      REITBENCH03        Total Return  US Dollar          USD   
1      REITBENCH03        Total Return  US Dollar          USD   
2      REITBENCH03        Total Return  US Dollar          USD   
3      REITBENCH03        Total Return  US Dollar          USD   
4      REITBENCH03        Total Return  US Dollar          USD   
...            ...                 ...        ...          ...   
2686   REITBENCH02        Total Return  US Dollar          USD   
2687   REITBENCH02        Total Return  US Dollar          USD   
2688   REITBENCH02        Total Return  US Dollar          USD   
2689   REITBENCH02        Total Return  US Dollar          USD   
2690   REITBENCH02        Total Return  US Dollar          USD   

     PERFORMANCEFREQUENCY               HISTORYDATE HISTORYDATE1       VALUE  
0     

## Validation Code

In [8]:
import logging
import pandas as pd

def validate_benchmark_performance(df: pd.DataFrame) -> pd.DataFrame:
    """
    Validate the BENCHMARKPERFORMANCE table before inserting into Snowflake.

    This validator checks for expected structure, nulls in key fields,
    valid performance types, and flags extreme values for proxy NAV returns
    and price data. It returns a summary DataFrame and logs all issues.

    Parameters
    ----------
    df : pd.DataFrame
        Performance-level REIT benchmark data with expected fields:
        BENCHMARKCODE, PERFORMANCEDATATYPE, CURRENCYCODE, CURRENCY,
        PERFORMANCEFREQUENCY, VALUE, HISTORYDATE.

    Returns
    -------
    pd.DataFrame
        One-row summary with validation flags and issue counts.
    """

    required_cols = [
        "BENCHMARKCODE", "PERFORMANCEDATATYPE", "CURRENCYCODE", "CURRENCY",
        "PERFORMANCEFREQUENCY", "VALUE", "HISTORYDATE"
    ]

    # Confirm required columns
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        logging.error(f"[benchmark_performance] Missing required columns: {missing_cols}")
        return pd.DataFrame()

    # Set of valid performance types (added "Total Return")
    valid_types = {"Proxy NAV Return", "Prices", "Total Return"}

    # Summary flags
    summary = {
        "missing_benchmarkcodes": df["BENCHMARKCODE"].isnull().sum(),
        "missing_values": df["VALUE"].isnull().sum(),
        "invalid_performance_types": (~df["PERFORMANCEDATATYPE"].isin(valid_types)).sum(),
        # Adjust outlier ranges as appropriate for Total Return data
        "nav_return_outliers": df.loc[
            df["PERFORMANCEDATATYPE"] == "Proxy NAV Return"
        ]["VALUE"].apply(lambda x: not (-1 <= x <= 2)).sum(),
        "total_return_outliers": df.loc[
            df["PERFORMANCEDATATYPE"] == "Total Return"
        ]["VALUE"].apply(lambda x: x <= 0).sum(),  # Total Return % usually positive, but you can adjust logic
        "price_outliers": df.loc[
            df["PERFORMANCEDATATYPE"] == "Prices"
        ]["VALUE"].apply(lambda x: x <= 0).sum()  # Prices shouldn't be <= 0
    }

    # Log issues
    for key, val in summary.items():
        if val:
            logging.warning(f"[benchmark_performance] {key.replace('_', ' ').capitalize()}: {val}")

    if not any(summary.values()):
        logging.info("[benchmark_performance] Validation passed.")

    return pd.DataFrame([summary])


## Populating BP Table in Snowflake

In [10]:
from snowflake.connector import connect
from snowflake.connector.pandas_tools import write_pandas
import os
import pandas as pd

def upload_and_merge_benchmark_performance(bp_df):
    """
    Uploads benchmark performance data (bp_df) to Snowflake staging table and merges into the
    target BENCHMARKPERFORMANCE table, handling duplicate rows by keeping only one record per key.

    Args:
        bp_df (pd.DataFrame): DataFrame containing benchmark performance data with columns:
            BENCHMARKCODE, PERFORMANCEDATATYPE, CURRENCYCODE, CURRENCY, PERFORMANCEFREQUENCY,
            VALUE, HISTORYDATE (timestamp), HISTORYDATE1 (date).

    Steps:
    1. Remove timezone info from HISTORYDATE for Snowflake compatibility.
    2. Connect to Snowflake using environment variables for credentials.
    3. Create or replace a temporary staging table with the appropriate schema.
    4. Upload the bp_df data into the staging table using Snowflake's write_pandas utility.
    5. Merge data from staging into BENCHMARKPERFORMANCE table with deduplication logic
       using ROW_NUMBER() to avoid duplicate merge conflicts.
    """
    # Step 1: Ensure HISTORYDATE is timezone-naive to match Snowflake TIMESTAMP_NTZ
    bp_df["HISTORYDATE"] = pd.to_datetime(bp_df["HISTORYDATE"]).dt.tz_localize(None)

    # Step 2: Establish connection to Snowflake
    conn = connect(
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        role=os.getenv("SNOWFLAKE_ROLE"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        database="AST_REALESTATE_DB",
        schema="DBO"
    )

    with conn.cursor() as cur:
        # Step 3: Set context and create temporary staging table
        cur.execute("USE DATABASE AST_REALESTATE_DB")
        cur.execute("USE SCHEMA DBO")

        cur.execute("""
            CREATE OR REPLACE TEMP TABLE STG_BENCHMARKPERFORMANCE (
                BENCHMARKCODE VARCHAR,
                PERFORMANCEDATATYPE VARCHAR,
                CURRENCYCODE VARCHAR,
                CURRENCY VARCHAR,
                PERFORMANCEFREQUENCY VARCHAR,
                VALUE FLOAT,
                HISTORYDATE TIMESTAMP_NTZ,
                HISTORYDATE1 DATE
            )
        """)

    # Step 4: Upload DataFrame to staging table
    success, nchunks, nrows, _ = write_pandas(
        conn=conn,
        df=bp_df,
        table_name="STG_BENCHMARKPERFORMANCE",
        quote_identifiers=False,
        use_logical_type=True  # Ensures correct data type handling
    )
    print(f"Uploaded {nrows} rows to STG_BENCHMARKPERFORMANCE")

    # Step 5: Perform MERGE with deduplication using ROW_NUMBER()
    with conn.cursor() as cur:
        cur.execute("""
            MERGE INTO BENCHMARKPERFORMANCE AS target
            USING (
                SELECT *
                FROM (
                    SELECT *,
                        ROW_NUMBER() OVER (
                            PARTITION BY BENCHMARKCODE, PERFORMANCEDATATYPE, CURRENCYCODE, PERFORMANCEFREQUENCY, HISTORYDATE
                            ORDER BY VALUE DESC
                        ) AS rn
                    FROM STG_BENCHMARKPERFORMANCE
                )
                WHERE rn = 1
            ) AS source
            ON target.BENCHMARKCODE = source.BENCHMARKCODE
               AND target.PERFORMANCEDATATYPE = source.PERFORMANCEDATATYPE
               AND target.CURRENCYCODE = source.CURRENCYCODE
               AND target.PERFORMANCEFREQUENCY = source.PERFORMANCEFREQUENCY
               AND target.HISTORYDATE = source.HISTORYDATE

            WHEN MATCHED THEN UPDATE SET
                target.VALUE = source.VALUE,
                target.CURRENCY = source.CURRENCY,
                target.HISTORYDATE1 = source.HISTORYDATE1

            WHEN NOT MATCHED THEN INSERT (
                BENCHMARKCODE, PERFORMANCEDATATYPE, CURRENCYCODE, CURRENCY,
                PERFORMANCEFREQUENCY, VALUE, HISTORYDATE, HISTORYDATE1
            ) VALUES (
                source.BENCHMARKCODE, source.PERFORMANCEDATATYPE, source.CURRENCYCODE, source.CURRENCY,
                source.PERFORMANCEFREQUENCY, source.VALUE, source.HISTORYDATE, source.HISTORYDATE1
            )
        """)
        print("MERGE into BENCHMARKPERFORMANCE completed.")

# Example usage:
# upload_and_merge_benchmark_performance(bp_df)


Uploaded 2691 rows to STG_BENCHMARKPERFORMANCE
MERGE into BENCHMARKPERFORMANCE completed.


# **Data Flow & Architecture Mapping**

## Source to Storage Mapping

| Stage              | Description                                                                                               |
| ------------------ | --------------------------------------------------------------------------------------------------------- |
| **Source**         | Yahoo Finance historical data pulled dynamically using tickers from `BenchmarkGeneralInformation`         |
| **Extraction**     | Python script uses `yfinance.Ticker.history()` to fetch adjusted close prices for benchmarks over time    |
| **Transformation** | Normalize data to daily total return index starting at 100; add metadata columns like currency, frequency |
|  • Normalize       | `VALUE = (Adj Close / first Adj Close) * 100` — relative total return index                               |
|  • Add fields      | `PERFORMANCEDATATYPE="Total Return"`, `CURRENCY="US Dollar"`, `CURRENCYCODE="USD"`, etc.                  |
|  • Date fields     | Includes both `HISTORYDATE` (Timestamp\_NTZ) and `HISTORYDATE1` (Date)                                    |
| **Validation**     | `validate_benchmark_performance()` checks required columns, missing values, invalid types, outliers       |
| **Storage**        | Upload via `write_pandas()` into staging table then MERGE into `BenchmarkPerformance`                     |
| **Consumption**    | Used for performance reporting, benchmark tracking, portfolio analysis, UI visualizations                 |


## Data Flow Diagram

          +---------------------------+
          | BenchmarkGeneralInformation |
          | (Symbols from Snowflake)    |
          +---------------------------+
                       |
               [Loop over each symbol]
                       |
                       v
          +---------------------------+
          | yfinance.Ticker.history() |
          | (Adj Close daily prices)  |
          +---------------------------+
                       |
                       v
          +---------------------------+
          | Normalize to index = 100  |
          | Add metadata fields       |
          +---------------------------+
                       |
                       v
          +---------------------------+
          | validate_benchmark_performance() |
          +---------------------------+
                       |
                       v
          +---------------------------+
          | write_pandas() staging    |
          | MERGE into BenchmarkPerformance |
          +---------------------------+
                       |
                       v
          +---------------------------+
          | Reporting, UI, analytics  |
          +---------------------------+


## Transformation Logic Documentation

### Value

| Transformation      | Description                                             |
| ------------------- | ------------------------------------------------------- |
| Index normalization | `VALUE = (Adj Close / first Adj Close) * 100`           |
| Purpose             | Converts prices into total return index starting at 100 |
| Data Type           | `FLOAT`                                                 |


### PerformanceFrequency

| Transformation | Description                     |
| -------------- | ------------------------------- |
| Fixed          | `"Daily"`                       |
| Purpose        | Frequency of price observations |
| Data Type      | `VARCHAR`                       |


### PerformanceDataType

| Transformation | Description                                  |
| -------------- | -------------------------------------------- |
| Fixed          | `"Total Return"`                             |
| Purpose        | Indicates data represents total return index |
| Data Type      | `VARCHAR`                                    |


### Date fields

| Field          | Description                                     | Data Type       |
| -------------- | ----------------------------------------------- | --------------- |
| `HISTORYDATE`  | Timestamp without timezone of each price record | `TIMESTAMP_NTZ` |
| `HISTORYDATE1` | Date portion extracted from `HISTORYDATE`       | `DATE`          |


### Currency / Currency Code

| Transformation | Description                              |
| -------------- | ---------------------------------------- |
| Fixed          | `"US Dollar"` / `"USD"`                  |
| Purpose        | Currency context of the benchmark prices |
| Data Type      | `VARCHAR`                                |


### Validation

| Check                       | Description                                                                          |
| --------------------------- | ------------------------------------------------------------------------------------ |
| Required columns            | `BENCHMARKCODE`, `VALUE`, `HISTORYDATE`, `PERFORMANCEDATATYPE`, etc. must be present |
| Missing values              | Nulls in `VALUE` or key fields flagged                                               |
| Performance type validation | Only accept `"Total Return"` or `"Price"`                                            |
| Outlier detection           | Flags `VALUE` outside expected range (e.g., negative or extreme)                     |
| Logging                     | Uses Python `logging` for warnings and errors                                        |


## Future Enhancements

| Feature                     | Description                                                        |
| --------------------------- | ------------------------------------------------------------------ |
| Multi-frequency support     | Add weekly/monthly aggregations                                    |
| Currency conversion         | Handle benchmarks priced in other currencies                       |
| Historical adjustments      | Factor in dividends, splits, or rebalancing events                 |
| Automated anomaly detection | More advanced outlier detection with machine learning              |
| CI/CD integration           | Automate validation and ingestion as part of ETL pipelines         |
| Incremental updates         | Support incremental data loads rather than full historical reloads |
